In [12]:
library(tidyverse)
library(janitor)


Attaching package: ‘janitor’




The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [30]:
stadt_data <- read_tsv("Open-Data-Bundestagswahl1343_wue_stadt.csv")
land_data <- read_tsv("Open-Data-Bundestagswahl1343_wue_land.csv")
votes <- read_tsv("okvote-fields.tsv")

Rows: 13 Columns: 71


── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): datum, wahl, gebiet-name
dbl (67): ags, max-schnellmeldungen, anz-schnellmeldungen, A1, A2, A3, A, B,...
lgl  (1): gebiet-nr



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 52 Columns: 71


── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): datum, wahl, gebiet-name
dbl (67): ags, max-schnellmeldungen, anz-schnellmeldungen, A1, A2, A3, A, B,...
lgl  (1): gebiet-nr



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 70 Columns: 3


── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): column, stimme, partei_beschreibung



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
head(stadt_data)

datum,wahl,ags,gebiet-nr,gebiet-name,max-schnellmeldungen,anz-schnellmeldungen,A1,A2,A3,⋯,D23,F23,D24,F24,D25,F25,D26,F26,D27,F27
<chr>,<chr>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,ALTSTADT,32,32,5959,9117,0,⋯,97,41,0,39,0,7,0,89,105,0
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,ZELLERAU,12,12,4313,4097,0,⋯,33,17,0,24,0,15,0,37,66,0
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,DÜRRBACHTAL,10,10,1936,2517,0,⋯,19,6,0,7,0,7,0,10,46,0
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,GROMBÜHL,10,10,2900,3188,0,⋯,29,14,0,12,0,9,0,28,36,0
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,LINDLEINSMÜHLE,4,4,1916,1386,0,⋯,5,2,0,10,0,3,0,4,28,0
26.09.2021,Wahl zum Deutschen Bundestag,9663000,NA,FRAUENLAND,26,26,6084,8374,0,⋯,66,34,0,24,0,12,0,79,102,0


In [7]:
head(votes)
tail(votes)

column,stimme,partei_beschreibung
<chr>,<chr>,<chr>
datum,NA,Datum des Wahltermins
wahl,NA,Name der Wahl
ags,NA,AGS der Behörde
gebiet-nr,NA,Nummer des Wahlgebiets
gebiet-name,NA,Name des Wahlgebiets
max-schnellmeldungen,NA,Anzahl an insgesamt erwarteten Schnellmeldungen im Wahlgebiet


column,stimme,partei_beschreibung
<chr>,<chr>,<chr>
F24,Zweitstimmen,Team Todenhöfer – Die Gerechtigkeitspartei
D25,Erststimmen,UNABHÄNGIGE für bürgernahe Demokratie
F25,Zweitstimmen,UNABHÄNGIGE für bürgernahe Demokratie
D26,Erststimmen,Volt Deutschland
F26,Zweitstimmen,Volt Deutschland
D27,Erststimmen,Einzelbewerber Schürer


In [35]:
stadt <- select(stadt_data, `gebiet-name`, D1:F27) %>%
    gather("Partei_Stimme", "Wert", 2:ncol(.)) %>% 
    left_join(votes, by = c("Partei_Stimme" = "column"))%>%
    filter(!is.na(stimme)) %>%
    mutate(Stadtbezirk = str_to_title(`gebiet-name`)) %>%
    select(-Partei_Stimme, -`gebiet-name`) %>%
    spread("stimme", "Wert") %>%
    replace(is.na(.), 0)

In [21]:
head(stadt)

partei_beschreibung,Stadtbezirk,Erststimmen,Zweitstimmen
<chr>,<chr>,<dbl>,<dbl>
Alternative für Deutschland,Altstadt,0,477
Alternative für Deutschland,Duerrbachtal,0,242
Alternative für Deutschland,Frauenland,0,520
Alternative für Deutschland,Grombuehl,0,202
Alternative für Deutschland,Heidingsfeld,0,405
Alternative für Deutschland,Heuchelhof,0,645


In [42]:
alter <- read_tsv("stadtbezirke_durchschnittsalter.tsv") %>%
    filter(Jahr==2019) %>%
    select(-Jahr) %>%
    mutate(
        Stadtbezirk=str_replace(Stadtbezirk, "Due", "Dü"),
        Stadtbezirk=str_replace(Stadtbezirk, "ueh", "üh"),
    )

Rows: 130 Columns: 3


── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): Stadtbezirk
dbl (2): Jahr, Durchschnittsalter



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [43]:
stadt_alter <- stadt %>% full_join(alter)

Joining, by = "Stadtbezirk"


In [44]:
unique(stadt_alter$Stadtbezirk)

[1] "Altstadt"       "Dürrbachtal"    "Frauenland"     "Grombühl"      
 [5] "Heidingsfeld"   "Heuchelhof"     "Lengfeld"       "Lindleinsmühle"
 [9] "Rottenbauer"    "Sanderau"       "Steinbachtal"   "Versbach"      
[13] "Zellerau"

In [85]:
land <- select(land_data, `gebiet-name`, D1:F27) %>%
    gather("Partei_Stimme", "Wert", 2:ncol(.)) %>% 
    left_join(votes, by = c("Partei_Stimme" = "column"))%>%
    filter(!is.na(stimme)) %>%
    mutate(Gemeinde = str_remove(`gebiet-name`, "Gemeinde ")) %>%
    mutate(Gemeinde = str_replace(Gemeinde, "b. Würzburg", "b.Würzburg")) %>%
    select(-Partei_Stimme, -`gebiet-name`) %>%
    spread("stimme", "Wert") %>%
    replace(is.na(.), 0)

In [55]:
land %>% pull(Gemeinde) %>% unique

[1] "Altertheim"                 "Bergtheim"                 
 [3] "Bieberehren"                "Eisingen"                  
 [5] "Erlabrunn"                  "Estenfeld"                 
 [7] "Gaukönigshofen"             "Gerbrunn"                  
 [9] "Geroldshausen"              "Greußenheim"               
[11] "Güntersleben"               "Hausen b. Würzburg"        
[13] "Hettstadt"                  "Holzkirchen"               
[15] "Kirchheim"                  "Kist"                      
[17] "Kleinrinderfeld"            "Kürnach"                   
[19] "Leinach"                    "Margetshöchheim"           
[21] "Oberpleichfeld"             "Prosselsheim"              
[23] "Riedenheim"                 "Rottendorf"                
[25] "Sonderhofen"                "Tauberrettersheim"         
[27] "Theilheim"                  "Thüngersheim"              
[29] "Uettingen"                  "Unterpleichfeld"           
[31] "Veitshöchheim"              "Waldbrunn"                 
[33] "Waldbüttelbrunn"            "Markt Bütthard"            
[35] "Markt Eisenheim"            "Markt Frickenhausen a.Main"
[37] "Markt Gelchsheim"           "Markt Giebelstadt"         
[39] "Markt Helmstadt"            "Markt Höchberg"            
[41] "Markt Neubrunn"             "Markt Randersacker"        
[43] "Markt Reichenberg"          "Markt Remlingen"           
[45] "Markt Rimpar"               "Markt Sommerhausen"        
[47] "Markt Winterhausen"         "Markt Zell a.Main"         
[49] "Stadt Aub"                  "Stadt Eibelstadt"          
[51] "Stadt Ochsenfurt"           "Stadt Röttingen"

In [74]:
land %>% pull(Gemeinde) %>% unique %>% length

[1] 52

In [89]:
oalter <- read_csv2("12421-106_clean.csv") %>%
    mutate(
        Gemeinde = str_replace(Gemeinde, "(.*), M$","Markt \\1"),
        Gemeinde = str_replace(Gemeinde, "(.*), St$","Stadt \\1"),
        Durchschnittsalter = `2019`
    ) %>%
    select(-id, -`2019`)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.



Rows: 2056 Columns: 3


── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (2): id, Gemeinde
dbl (1): 2019



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [90]:
oalter %>% filter(str_detect(Gemeinde, "Hausen"))

Gemeinde,Durchschnittsalter
<chr>,<dbl>
Hausen,42.1
Hausen,44.2
Hausen,48.0
Hausen,43.6
Hausen b.Würzburg,43.1


In [91]:
land_alter <- land %>% inner_join(oalter)

Joining, by = "Gemeinde"


In [112]:
data <- land_alter %>% 
    mutate(Stadt=FALSE) %>%
    rename(Ort=Gemeinde) %>%
    add_row(
        stadt_alter %>%
            mutate(Stadt=TRUE) %>%
            rename(Ort=Stadtbezirk)
    ) %>%
    rename(Partei=partei_beschreibung) %>%
    group_by(Ort) %>%
    mutate(
        ErststimmenAnteil = Erststimmen/sum(Erststimmen),
        ZweitstimmenAnteil = Zweitstimmen/sum(Zweitstimmen)
    ) %>%
    ungroup %>%
    select(Ort, Partei, Stadt, Durchschnittsalter, Erststimmen, ErststimmenAnteil, Zweitstimmen, ZweitstimmenAnteil)

In [116]:
sample_n(data, 10)

Ort,Partei,Stadt,Durchschnittsalter,Erststimmen,ErststimmenAnteil,Zweitstimmen,ZweitstimmenAnteil
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Markt Gelchsheim,Die Urbane. Eine HipHop Partei,FALSE,44.5,0,0.0000000,3,0.0052173913
Uettingen,Bündnis C - Christen für Deutschland,FALSE,43.5,0,0.0000000,0,0.0000000000
Stadt Röttingen,PARTEI MENSCH UMWELT TIERSCHUTZ,FALSE,47.0,0,0.0000000,12,0.0109689214
Heidingsfeld,Marxistisch-Leninistische Partei Deutschlands,TRUE,45.7,0,0.0000000,1,0.0001697793
Zellerau,Liberal-Konservative Reformer,TRUE,41.8,0,0.0000000,0,0.0000000000
Leinach,Christlich-Soziale Union in Bayern e.V.,FALSE,44.4,955,0.4513233,819,0.3836065574
Markt Helmstadt,DER DRITTE WEG,FALSE,44.0,0,0.0000000,0,0.0000000000
Versbach,Nationaldemokratische Partei Deutschlands,TRUE,44.6,0,0.0000000,2,0.0004596644
Markt Frickenhausen a.Main,DIE LINKE,FALSE,46.5,21,0.0260223,12,0.0148883375


In [117]:
data %>% write_tsv("btw2021_wuerzburg_age.tsv")